### Presimulation Notebook # 3 - DIGNAD Macroeconomic Model pre-simulation

Because each DIGNAD run takes ~1 min to complete, large Monte-Carlo simulations aren't really feasible. This notebook runs DIGNAD presimulations over a realistic parameter space to produce a dataset that can easily be incorporated into later Monte Carlo runs and significantly speed up those simulations. 

In this notebook, we run a flood risk simulation to understand get a range of possible losses. Using these results we create a parameter grid over which we will run the DIGNAD simulations. For an adaptation and no adaptation scenario we run ~1000 DIGNAD simulations (this takes ~16 hours per run) and save results to a CSV

In [1]:
# Import live code changes in
%load_ext autoreload
%autoreload 

from pathlib import Path
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path

from sovereign.flood import build_basin_curves, BasinLossCurve, risk_data_future_shift, run_simulation, extract_sectoral_losses
from sovereign.macroeconomic import run_flood_sim_for_macro, create_dignad_parameter_grid, prepare_DIGNAD, run_DIGNAD

##### 1. User config

In [2]:
adaptation_aep = 0.01 # 100-year flood protection (adaptation scenario)
n_years = 100000 # number of years to simulate (wouldn't normally do 100K but want to get worst-case)
# Future climate parameters (we want max risk so choose latest epoch, highest emission scenario and q95)
future_hydro = 'jules-w2'
future_epoch = 2070
future_scenario = 'ssp585'
future_stat = 'q95'
# Macroeconomic data (for preparing DIGNAD simulation)
Thai_GDP = 496e9 # 2022 numbers in USD
# National GVA figures from DOSE
agr_GVA = 42880325598
man_GVA = 162659433017
ser_GVA = 316647741231
# Disaggregate output losses (what share of each sector are tradable)
TRADABLE_SHARES = {
    "Agriculture": 1.0,
    "Manufacturing": 0.7,
    "Service": 0.5,
}
# DIGNAD parameters
n_dignad_simulations = 1000 # takes ~ 16 hours but gives good coverage of parameter space
sim_start_year = 2022
nat_disaster_year = 2027 # disaster happens n+5 years after start of simulation
recovery_period = 3 # recover time in DIGNAD
adaptation_cost = 21.97 # billion (can find the cost of the adaptation scenario in the preparation/flood_protection.ipynb notebook)
reconstruction_efficiency = 0 # non-adjustable parameter
public_debt_premium = 0 # non-adjustable parameter
gdp_avg_years = 5 # we are intereseted in calculating average GDP impact over this period

##### 2. Set filepaths and load data

In [3]:
root = Path.cwd().parent.parent # find project root
THA_calibration_path = os.path.join(root, "inputs", "macro", "THA_2022_calibration_final.csv") # DIGNAD calibration
risk_basin_path = os.path.join(root, 'outputs', 'flood', 'risk', 'basins', 'risk_basins.csv')
copula_path = os.path.join(root, 'outputs', 'flood', 'dependence', 'copulas')
copula_random_numbers = pd.read_parquet(os.path.join(copula_path, "copula_random_numbers.gzip"))
dignad_output_noadapt = os.path.join(root, 'outputs', 'macro', f'DIGNAD_presim_n{n_dignad_simulations}_noadapt.csv')
dignad_output_adapt = os.path.join(root, 'outputs', 'macro', f'DIGNAD_presim_n{n_dignad_simulations}_adapt.csv')
risk_data = pd.read_csv(risk_basin_path)
future_rp_shifts = pd.read_csv(os.path.join(root, 'outputs', 'flood', 'future', 'basin_rp_shifts.csv'))
# Fix the risk data
# Drop first "unnamed column"
risk_data = risk_data.iloc[:, 1:]
# Add AEP column
risk_data['AEP'] = 1 / risk_data['RP']
# Add a column converting current prorection level into AEP
risk_data['Pr_L_AEP'] = np.where(risk_data['Pr_L'] == 0, 0, 1 / risk_data['Pr_L']) # using numpy where avoids zero division errors
risk_data.reset_index(drop=True, inplace=True)

##### 3. Prepare data for risk assessment (future climate adjustment and loss curve creation)

In [4]:
# Adjust future risk data
future_risk_data = risk_data_future_shift(risk_data, future_rp_shifts, future_hydro, future_scenario, future_epoch, future_stat, degrade_protection=True)
# Build basin loss probability curves
baseline_curves: dict[int, BasinLossCurve] = build_basin_curves(risk_data)
future_curves: dict[int, BasinLossCurve] = build_basin_curves(future_risk_data)

##### 4. Run flood risk assessment (outputting DIGNAD inputs)

In [5]:
# Baseline (no climate change)
baseline_current_df, baseline_adapted_df = run_flood_sim_for_macro(
    baseline_curves, adaptation_aep, n_years, copula_random_numbers,
    agr_GVA, man_GVA, ser_GVA, TRADABLE_SHARES, Thai_GDP
)
# Future (with climate change)
future_current_df, future_adapted_df = run_flood_sim_for_macro(
    future_curves, adaptation_aep, n_years, copula_random_numbers,
    agr_GVA, man_GVA, ser_GVA, TRADABLE_SHARES, Thai_GDP
)

100%|█████████████████████████████████████████████████████████████████████████| 100000/100000 [11:06<00:00, 150.05it/s]


In [6]:
# Combine all scenarios into one DataFrame (only need to use no adaptation, as losses will be higher)
flood_combined = pd.concat([baseline_current_df, future_current_df], ignore_index=True)

##### 5. Create DIGNAD parameter grid
In this step we create a parameter grid of DIGNAD shock combinations from our flood simulation results. We do this by sampling the joint parameter space using either k-means cluster centres, stratified sampling by event severity, KDE-based resampling, or a combination of all three. We also append tail events to ensure we are capturing edge cases in parameter grid

In [7]:
param_grid = create_dignad_parameter_grid(
    flood_combined,
    n_samples=20,
    method='combined',  # Uses multiple sampling methods
)


Creating DIGNAD parameter grid using 'combined' method...
Target samples: 20

Final parameter grid:
  Grid size: 42 unique combinations
  Parameter ranges:
    dY_T: [0.000, 0.251]
    dY_N: [0.000, 0.284]
    dK_priv: [0.000, 0.467]
    dK_pub: [0.000, 0.168]


##### 6. Run DIGNAD pre-simulation (no adaptation)

In [8]:
# Calibrate DIGNAD (baseline = no adaptation)
print('Calibrating DIGNAD (no adaptation)')
prepare_DIGNAD(THA_calibration_path, adaptation_cost=0, root_dir=root)

# Run pre-compuataion
results_na = []
failure_index_na = []
for row in tqdm(param_grid.itertuples(index=False), total=len(param_grid), desc="Running DIGNAD precomputation"):
    # Extract shock paramaters
    index = row.grid_index
    tradable_impact = row.tradable_impact
    nontradable_impact = row.nontradable_impact
    private_impact = row.private_impact
    public_impact = row.public_impact
    share_tradable = row.share_tradable
    reconstruction_efficiency = row.reconstruction_efficiency
    public_debt_premium = row.public_debt_premium

    # If shocks are 0 don't run DIGNAD
    if tradable_impact == 0 and nontradable_impact == 0 and private_impact == 0 and public_impact == 0:
        results_na.append({
            "index": index,
            "gdp_avg": 0
        })
        continue

    # Run DIGNAD
    gdp_impact, years = run_DIGNAD(sim_start_year, nat_disaster_year, recovery_period, tradable_impact, nontradable_impact,
                                    reconstruction_efficiency, public_debt_premium, public_impact, private_impact, share_tradable, root)

    if gdp_impact is None:
        # Means MatLab failed to execute
        failure_index_na.append(index)
        gdp_avg = None
    else:
        t_shock = nat_disaster_year-sim_start_year
        gdp_avg = np.mean(gdp_impact[t_shock : t_shock + gdp_avg_years])

    results_na.append({
        "index": index,
        "gdp_avg": gdp_avg
    })
results_df_na = pd.DataFrame(results_na)

# Combine with parameter grid and prepare dataframe for saving
precomputed_results_na = param_grid.merge(results_df_na, left_on="grid_index", right_on="index")
precomputed_results_na = precomputed_results_na[precomputed_results_na['gdp_avg'].notna()] # remove NaN / failed

# Rename columns to match simulation outputs
precomputed_results_r_na = precomputed_results_na.copy()
precomputed_results_r_na.rename(columns={
        'tradable_impact': 'dY_T',
        'nontradable_impact': 'dY_N', 
        'private_impact': 'dK_priv',
        'public_impact': 'dK_pub'
    }, inplace=True)

# Save to CSV
Path(dignad_output_adapt).parent.mkdir(parents=True, exist_ok=True) # create directory if it doesn't already exist
precomputed_results_r_na.to_csv(dignad_output_adapt)

Calibrating DIGNAD (no adaptation)


Running DIGNAD precomputation:  17%|████████▋                                           | 7/42 [06:42<32:09, 55.14s/it]

MATLAB script not executed succesfully


Running DIGNAD precomputation: 100%|███████████████████████████████████████████████████| 42/42 [39:20<00:00, 56.20s/it]


##### 7. Run DIGNAD pre-simulation (adaptation)

In [ ]:
# Calibrate DIGNAD (baseline = no adaptation)
print('Calibrating DIGNAD (adaptation)')
prepare_DIGNAD(THA_calibration_path, adaptation_cost=adaptation_cost, root_dir=root)

# Run pre-compuataion
results_a = []
failure_index_a = []
for row in tqdm(param_grid.itertuples(index=False), total=len(param_grid), desc="Running DIGNAD precomputation"):
    # Extract shock paramaters
    index = row.grid_index
    tradable_impact = row.tradable_impact
    nontradable_impact = row.nontradable_impact
    private_impact = row.private_impact
    public_impact = row.public_impact
    share_tradable = row.share_tradable
    reconstruction_efficiency = row.reconstruction_efficiency
    public_debt_premium = row.public_debt_premium

    # If shocks are 0 don't run DIGNAD
    if tradable_impact == 0 and nontradable_impact == 0 and private_impact == 0 and public_impact == 0:
        results_a.append({
            "index": index,
            "gdp_avg": 0
        })
        continue

    # Run DIGNAD
    gdp_impact, years = run_DIGNAD(sim_start_year, nat_disaster_year, recovery_period, tradable_impact, nontradable_impact,
                                    reconstruction_efficiency, public_debt_premium, public_impact, private_impact, share_tradable, root)

    if gdp_impact is None:
        # Means MatLab failed to execute
        failure_index_a.append(index)
        gdp_avg = None
    else:
        t_shock = nat_disaster_year-sim_start_year
        gdp_avg = np.mean(gdp_impact[t_shock : t_shock + gdp_avg_years])

    results_a.append({
        "index": index,
        "gdp_avg": gdp_avg
    })
results_df_a = pd.DataFrame(results_a)

# Combine with parameter grid and prepare dataframe for saving
precomputed_results_a = param_grid.merge(results_df_a, left_on="grid_index", right_on="index")
precomputed_results_a = precomputed_results_a[precomputed_results_a['gdp_avg'].notna()] # remove NaN / failed

# Rename columns to match simulation outputs
precomputed_results_r_a = precomputed_results_a.copy()
precomputed_results_r_a.rename(columns={
        'tradable_impact': 'dY_T',
        'nontradable_impact': 'dY_N', 
        'private_impact': 'dK_priv',
        'public_impact': 'dK_pub'
    }, inplace=True)

# Save to CSV
Path(dignad_output_adapt).parent.mkdir(parents=True, exist_ok=True) # create directory if it doesn't already exist
precomputed_results_r_a.to_csv(dignad_output_adapt)

Calibrating DIGNAD (adaptation)


Running DIGNAD precomputation:  17%|████████▋                                           | 7/42 [06:34<31:51, 54.60s/it]

MATLAB script not executed succesfully


Running DIGNAD precomputation:  57%|████████████████████████████▌                     | 24/42 [29:45<30:12, 100.68s/it]